### fMRI mini project - Time_Frequency analysis for resting state maps
In this project, we will work on fMRI data from the  Amsterdam Open MRI Collection (AOMIC).

The AOMIC dataset gathers MRI data from more than a thousand individuals obtained on a 3 Tesla imager. For each subject we can access the T1-weighted images ( anatomical image), the diffusion-weighted images ( white-matter tracts)  and fMRI sequences (task-based and resting states). The dataset gives access to both raw and preprocessed (derivative) data. The description of the data acquisition and processing is available here : 

**Snoek, L., van der Miesen, M. M., Beemsterboer, T., Van Der Leij, A., Eigenhuis, A., & Scholte, H. S. (2021). The Amsterdam Open MRI Collection, a set of multimodal MRI datasets for individual difference analyses. Scientific data, 8(1), 1-23.**

All data are publicly available for downloads using AWS s3 buckets s3://openneuro.org/. The projects will use Jupyter Notebook with the following library : numpy, scipy,  scikit-learn; nilearn.

We will use time-frequency analysis to propose connectivity map in resting-state and compared with the RSN proposed in the paper :   **Smith, SM, Fox, PT, Miller, KL, Glahn, DC, Fox, PM, Mackay, CE, Filippini, N, Watkins, KE, Toro, R, Laird, AR, Beckmann, CF (2009). Correspondence of the brain's functional architecture during activation and rest. Proc Natl Acad Sci U S A, 106, 31:13040-5.**


 
 

In [ ]:
###############################
# Installing required libraries
 
%matplotlib inline
!pip install nilearn
!pip install datalad[full]
#!pip install git-annex # might be needed
import os
import nilearn 
import re 
import numpy as np
from nilearn import datasets
import pandas as pd

From the S3 bucket "openneuro.org"  all links available in https://github.com/OpenNeuroDatasets/dsxxxx. The file and directory strucure corresponds to the path of the object in a [BIDS](https://bids-specification.readthedocs.io/en/stable/index.html) standard.

Find the dataset for  PIOP1 cohort (it starts with dsXXX....)



In [ ]:
!datalad clone https://github.com/OpenNeuroDatasets/d.....



PIOP1 cohort is in directory dsxxx 

preprocessed data are in /derivatives folder

let's get all the task working memory fmri files for  individual sub-001

we are looking for 'task-restingstate' in the files

In [ ]:
#!datalad clone https://github.com/OpenNeuroDatasets/ds002790.git
import glob
dir_nii=...
task_list=glob.glob(dir_nii+...)
task_list

In [ ]:
# Run through all files with nii.gz endings
import pandas as pd
nii_task=np.array([x.endswith('nii.gz') for x in task_list]) 
nii_files=np.asarray(task_list)[nii_task]
nii_files

now list all the nii.gz files

we  can use 'endwith' function 

In [ ]:
# 2 - Use Panda library to select our files

import pandas as pd
dir_nii="...
task_list=os.listdir(dir_nii)

nii_task=np.array([x.endswith(....) for x in task_list]) 
nii_files=np.asarray(task_list)[nii_task]
nii_files

Select the file for preprocessed  frmi volumes in MNI space and dowload it with datalad

In [ ]:
! cd ds002790 ; datalad get ...

In [ ]:
resting_bold=...


In [ ]:
from nilearn import plotting
from nilearn import image

img_path=dir_nii+"/"+resting_bold
img_path
img= ... # load your image with image.load_img
print(img.get_data().shape) # check image dimensions with shape
# launch interactive 3d view with view_img function
plotting.view_img(image.index_img(img, 0))

In [ ]:
# Now we going to compute the mean (average) image usinng numpy
# you need to import the numpy library
import ...
# find nilearn image function to compute average 3d volume of a fMRI sequence volume
# compute average image using image library
mean_img= ...
# always check the dimension of the data 
print(.... ) 
# use the  triplanar interactive view of nilearn
# to explore the average 3d volume
... mean_img ...  

Now we are going to extract all the data and plot them separately either as 2d image eaither aas time series.

In [ ]:

from matplotlib import pyplot as plt

# extract 4D array from nilear image object
volume= ....
# check the dimension of the data 
print (volume.shape)
# eextract one slice (2D image)
flat_slice=volume..... 
# check thye size
print (flat_slice.shape)
# use matplotlib imshow to plot the slice  
plt.imshow(flat_slice)


In [ ]:
# extract the time serie for one voxel 
ts=volume ....
# in the AOMICS website find repeat tim RT RT : time in sec between 2 image in [s]  
dt=...
# make a vector of slice times in [s]  
time_vec= ...# vector 
print(time_vec)

In [ ]:
plt.figure(figsize=(14,7))
# plot the normalaized time serie
plt.plot(... , label='bold signal at'+np.array2string(np.array([30,35,30])), linewidth=0.5)
plt.ylabel('bold signal')
plt.title('bold signal for 1 voxel')
plt.xlabel('time in [s]')
# Add legend
plt.legend()
# Auto space
plt.tight_layout()
# Display plot
plt.show() 

In the AOMICS website find which file is the segmentation file (the brain regions) and dowload it with datalad.

We will use  the aparc + aseg segmentation image

The regions  labels are detailed here :
 https://github.com/freesurfer/freesurfer/blob/dev/distribution/FreeSurferColorLUT.txt

In [ ]:
! cd ds002790 ; datalad get ...

In [ ]:
file_key = ...  
file_path=dir_nii+"/"+file_key
img_seg=image.load_img(file_path)
print(np.asanyarray(img_seg.dataobj).shape)
plotting.plot_roi(img_seg, threshold=1, cmap=plt.cm.prism)
plotting.view_img(img_seg)

In [ ]:
from nilearn.masking import apply_mask
import nibabel as nb
img_segbool=np.asanyarray(img_seg.dataobj)#
img_segbool[:]=0
for i_seg in [...]: #find one or several  region of interest as the seed for Default mode network
    mask=np.where(img_array==i_seg)
    img_segbool[mask]=10

mask
#img_segbool[0:,0:,0:]=0
print(np.sum(img_segbool)/10)
nii_img_seg=nb.Nifti1Image(img_segbool, affine=img_seg.affine)
plotting.plot_roi(nii_img_seg, threshold=1, cmap=plt.cm.prism)
plotting.view_img(nii_img_seg)

In the following we will use the segmentation to create a mask of region to extract all the time series from one region 

In [ ]:
from nilearn.masking import apply_mask
import nibabel as nb
# extract the segmentation volume in a np.array 
img_segbool=np.array(..)
# always check the size
print(np.sum(img_segbool))
# with np.where find the voxels of the region of interest 
# that will bu used as seed for Default Mode Network
mask1=np.where(...)
# now set all background to 0
img_segbool....=0
#  and set the voxels from the mask to 10
img_segbool...=10


#always check the size is right
img_segbool.shape
# count hozw many voxels in the region with np.sum
np.sum(...)
#  find how to make a proper nii volume from img_segbool 
#  using  Nifti1Image function and the original img_seg volume 
nii_img_seg=nb.Nifti1Image(....)
# use the interactive region viewer to check your segmentation 
.....
# find how to use apply_mask to retreive the timeseries of the selected region
# using the original nii volume img and the region volume nii_img_seg
masked_data = ....

# check masked_data shape is (timepoints, voxels).
masked_data....

In [ ]:
# And now plot two time series
import matplotlib.pyplot as plt
plt.figure(figsize=(10, 3))
plt.plot(masked_data[:, :2])
plt.xlabel('Time [TRs]', fontsize=16)
plt.ylabel('Intensity', fontsize=16)
#plt.xlim(0, 150)
plt.subplots_adjust(bottom=.12, top=.95, right=.95, left=.12)

plt.show()

Now we are going to process the time serie. 
First we need to load the scipy libraries

In [ ]:
from scipy.fft import fft,fftfreq
from scipy.fftpack import fftshift
from scipy.signal import spectrogram

## First step  is removing complex signal drift
For that we are fititng three degree polygone curve.

In [ ]:
### removing complex signal drift 
#choose a time serie as example : 
ts = ...
# a / fit a 4 degree polynome -- or a low frequency cosine
# for a polynome
# find a0, a, b, c, d that fit the signal y= a0 + ax + bx² +cx³ +dx⁴
# you can use the code used in the filtering exercice 
# with optimize.curve_fit and a test_func 
# or use numpy.polyfit
# for low frequency cosine, use very low (<0.01) frequecy
#for starting parameters optimize.curve_fit  

drift = ....

# b/ remove the fitted drift from the signa
yf=ts-drift


In [ ]:
# in the AOMICS website find repeat tim RT RT : time in sec between 2 image in [s]  
dt=...
# make a vector of slice times in [s]  
time_vec= ...# vector 
print(time_vec)

In [ ]:
#plotting the whole process
t = time_vec
y=ts
plt.figure(figsize=(14,10))

# plot signal and the fitted drift
ax2=plt.subplot(411)
plt.plot(t, y, 'b-', label='signal')
plt.plot(t,drift, 'g--', label='drift')
plt.ylabel('bold signal')
plt.legend()
plt.grid()

# plot  sprectrogram for original signal 
freqs, times, spectro = spectrogram( y,fs=1/dt, nperseg=3)
plt.subplot(412, sharex=ax2)
plt.pcolormesh(times, fftshift(freqs), fftshift(spectro, axes=0), shading='gouraud')
plt.ylabel('f [Hz]')
plt.legend()

# plot old and new signal (centerd on the mean)
plt.subplot(413, sharex=ax2)
plt.plot(np.array(range(0,len(ts)))*dt, y-y.mean(), label='old bold signal', linewidth=0.5)
plt.plot(np.array(range(0,len(ts)))*dt, yf-yf.mean(), label='new voxel bold signal', linewidth=2)
plt.legend()

# plot  sprectrogram for new signal 
freqs, times, spectro = spectrogram( yf,fs=1/dt, nperseg=3)
plt.subplot(414, sharex=ax2)
plt.pcolormesh(times, fftshift(freqs), fftshift(spectro, axes=0), shading='gouraud')
plt.ylabel('f [Hz]')
plt.xlabel('t [sec]')
plt.legend()

# Auto space
plt.tight_layout()

# Display plot
plt.show()

In [ ]:

# Make a high pass filter with the code use in the filtering exercice 
# find the value in the reference article
fc = ... # desired cutoff frequency of the filter, in Hz.
....
# Plot the frequency response ( the code use in the filtering exercice)
...
# calculate the FFT of the original signal and plot the frequency components (the code use in the filtering exercice)
...
# apply the filter to the signal obtained after drift removal (the code use in the filtering exercice)
...
# plot both the original and filtered signals (the code use in the filtering exercice)
...
# calculate the FFT of the filtered signal and plot the frequency components (the code use in the filtering exercice)
...
# plot spectrogram (the code use in the previous cell )  before and after filtering

In [ ]:
# Make a low pass filter  with the code use in the filtering exercice
# find the value in the reference article
fc = ...
# Plot the frequency response ( the code use in the filtering exercice)
...
# calculate the FFT of the original signal and plot the frequency components (the code use in the filtering exercice)
...
# apply the filter to the signal obtained after drift removal (the code use in the filtering exercice)
...
# plot both the original and filtered signals (the code use in the filtering exercice)
...
# calculate the FFT of the filtered signal and plot the frequency components (the code use in the filtering exercice)
...
# plot spectrogram (the code use in the previous cell )  before and after filtering

In [ ]:

# write a function that implemets all the steps
# and propose a measure of correlation based ond frequecy spectrum  

In [ ]:
# extend to cross-region analysis using the seed methods